## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Current Weather,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Oksfjord,NO,70.2396,22.3479,light snow,27.00,86,90,12.66
1,1,Tessalit,ML,20.1986,1.0114,overcast clouds,68.63,22,100,8.72
2,2,Ushuaia,AR,-54.8000,-68.3000,broken clouds,50.00,76,75,12.66
3,3,Tanout,NE,14.9709,8.8879,overcast clouds,58.73,29,86,9.24
4,4,Hithadhoo,MV,-0.6000,73.0833,broken clouds,81.30,75,61,12.77


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Current Weather,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Hithadhoo,MV,-0.6000,73.0833,broken clouds,81.30,75,61,12.77
5,5,Avarua,CK,-21.2078,-159.7750,few clouds,86.00,66,17,11.50
9,9,Hilo,US,19.7297,-155.0900,broken clouds,78.80,53,75,5.75
11,11,Pemba,MZ,-12.9740,40.5178,overcast clouds,77.67,87,100,7.72
14,14,Rikitea,PF,-23.1203,-134.9692,broken clouds,78.78,69,73,10.56
16,16,Mataura,NZ,-46.1927,168.8643,clear sky,81.86,44,5,1.34
21,21,Puerto Ayora,EC,-0.7393,-90.3518,broken clouds,77.00,94,66,6.76
23,23,Butaritari,KI,3.0707,172.7902,scattered clouds,81.23,79,38,20.22
25,25,Moron,AR,-34.6534,-58.6198,clear sky,81.00,54,0,3.29
27,27,Carnarvon,AU,-24.8667,113.6333,scattered clouds,84.20,100,40,13.80


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID            162
City               162
Country            162
Lat                162
Lng                162
Current Weather    162
Max Temp           162
Humidity           162
Cloudiness         162
Wind Speed         162
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
4,Hithadhoo,MV,81.30,broken clouds,-0.6000,73.0833,
5,Avarua,CK,86.00,few clouds,-21.2078,-159.7750,
9,Hilo,US,78.80,broken clouds,19.7297,-155.0900,
11,Pemba,MZ,77.67,overcast clouds,-12.9740,40.5178,
14,Rikitea,PF,78.78,broken clouds,-23.1203,-134.9692,
16,Mataura,NZ,81.86,clear sky,-46.1927,168.8643,
21,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,
23,Butaritari,KI,81.23,scattered clouds,3.0707,172.7902,
25,Moron,AR,81.00,clear sky,-34.6534,-58.6198,
27,Carnarvon,AU,84.20,scattered clouds,-24.8667,113.6333,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
4,Hithadhoo,MV,81.30,broken clouds,-0.6000,73.0833,Scoop Guest House
5,Avarua,CK,86.00,few clouds,-21.2078,-159.7750,Paradise Inn
9,Hilo,US,78.80,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Pemba,MZ,77.67,overcast clouds,-12.9740,40.5178,Guesthouse Samotina
14,Rikitea,PF,78.78,broken clouds,-23.1203,-134.9692,Pension Maro'i
16,Mataura,NZ,81.86,clear sky,-46.1927,168.8643,Ellie's Villa
21,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Castillo Galapagos
23,Butaritari,KI,81.23,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
25,Moron,AR,81.00,clear sky,-34.6534,-58.6198,Hotel Morón
27,Carnarvon,AU,84.20,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
4,Hithadhoo,MV,81.30,broken clouds,-0.6000,73.0833,Scoop Guest House
5,Avarua,CK,86.00,few clouds,-21.2078,-159.7750,Paradise Inn
9,Hilo,US,78.80,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Pemba,MZ,77.67,overcast clouds,-12.9740,40.5178,Guesthouse Samotina
14,Rikitea,PF,78.78,broken clouds,-23.1203,-134.9692,Pension Maro'i
16,Mataura,NZ,81.86,clear sky,-46.1927,168.8643,Ellie's Villa
21,Puerto Ayora,EC,77.00,broken clouds,-0.7393,-90.3518,Castillo Galapagos
23,Butaritari,KI,81.23,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
25,Moron,AR,81.00,clear sky,-34.6534,-58.6198,Hotel Morón
27,Carnarvon,AU,84.20,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon


In [19]:
# 8a. Create the output File (CSV)
output_data_file= "../Vacation_Search/WeatherPy_vacatoin.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Weather}</dd>
<dt>Max Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))